# Bedrock Python Demo

The following is a demo of the Bedrock python package, used to ease the configuration of a Bedrock deployment using Python.

In [1]:
import sys
sys.path.append('/projects/mochi/py-bedrock') # use the local source of py-mochi-bedrock

## Getting started

The root of a Bedrock configuration is a `ProcSpec`, defined in the `bedrock.spec` package. It defines everything we want to run on a process.

In [2]:
import bedrock.spec

We can create a `ProcSpec` using a single `margo` key specifying the protocol we want to use.

In [3]:
my_process = bedrock.spec.ProcSpec(margo='na+sm')

All the specification objects have a `to_json()` method to generate their JSON configuration. Let's look at our `proc`'s configuration:

In [4]:
print(my_process.to_json(indent=4))

{
    "margo": {
        "progress_timeout_ub_msec": 100,
        "enable_profiling": false,
        "enable_diagnostics": false,
        "handle_cache_size": 32,
        "profile_sparkline_timeslice_msec": 1000,
        "argobots": {
            "abt_mem_max_num_stacks": 8,
            "abt_thread_stacksize": 2097152,
            "version": "unknown",
            "pools": [
                {
                    "name": "__primary__",
                    "kind": "fifo_wait",
                    "access": "mpmc"
                }
            ],
            "xstreams": [
                {
                    "name": "__primary__",
                    "cpubind": -1,
                    "affinity": [],
                    "scheduler": {
                        "type": "basic_wait",
                        "pools": [
                            "__primary__"
                        ]
                    }
                }
            ]
        },
        "mercury": {
            "address":

Py-bedrock gives us a very simple way of accessing sub-configurations. Objects that have a name can be accessed using their name or their index in the list that contains them. For example, accessing Argobots' primary pool:

In [5]:
print(my_process.margo.argobots.pools['__primary__'])
print(my_process.margo.argobots.pools[0])

PoolSpec(name='__primary__', kind='fifo_wait', access='mpmc')
PoolSpec(name='__primary__', kind='fifo_wait', access='mpmc')


Let's customize our process a little more. What about defining a pool for RPCs to run on, along with a few execution streams using that pool?

In [6]:
# Create the new pool, it will be added automatically to the process
my_rpc_pool = my_process.margo.argobots.pools.add(name='my_rpc_pool', kind='fifo', access='mpmc')
# Create two execution streams using that pool
for i in range(0,4):
    sched = bedrock.spec.SchedulerSpec(type='basic', pools=[my_rpc_pool])
    my_process.margo.argobots.xstreams.add(name=f'my_xstream_{i}', scheduler=sched)
# Now let's set the pools we want for handling RPCs
my_process.margo.rpc_pool = my_rpc_pool

Let's now print the resulting configuration:

In [7]:
print(my_process.to_json(indent=4))

{
    "margo": {
        "progress_timeout_ub_msec": 100,
        "enable_profiling": false,
        "enable_diagnostics": false,
        "handle_cache_size": 32,
        "profile_sparkline_timeslice_msec": 1000,
        "argobots": {
            "abt_mem_max_num_stacks": 8,
            "abt_thread_stacksize": 2097152,
            "version": "unknown",
            "pools": [
                {
                    "name": "__primary__",
                    "kind": "fifo_wait",
                    "access": "mpmc"
                },
                {
                    "name": "my_rpc_pool",
                    "kind": "fifo",
                    "access": "mpmc"
                }
            ],
            "xstreams": [
                {
                    "name": "__primary__",
                    "cpubind": -1,
                    "affinity": [],
                    "scheduler": {
                        "type": "basic_wait",
                        "pools": [
                       

## Adding ABT-IO instances

Let's add an ABT-IO instance.

In [8]:
my_process.abt_io.add(name='my_abtio', pool=my_rpc_pool)

AbtIOSpec(name='my_abtio', pool=PoolSpec(name='my_rpc_pool', kind='fifo', access='mpmc'))

## Adding SSG groups
Let's add an SSG group.

In [9]:
swim_config = bedrock.spec.SwimSpec(
    period_length_ms=1000,
    suspect_timeout_periods=3,
    subgroup_member_count=0,
    disabled=False)

my_process.ssg.add(name='my_group',
                   swim=swim_config,
                   bootstrap='init',
                   group_file='my_group.ssg',
                   pool=my_rpc_pool)

SSGSpec(name='my_group', pool=PoolSpec(name='my_rpc_pool', kind='fifo', access='mpmc'), credential=-1, bootstrap='init', group_file='my_group.ssg', swim=SwimSpec(period_length_ms=1000, suspect_timeout_periods=3, subgroup_member_count=0, disabled=False))

In [10]:
print(my_process.to_json(indent=4))

{
    "margo": {
        "progress_timeout_ub_msec": 100,
        "enable_profiling": false,
        "enable_diagnostics": false,
        "handle_cache_size": 32,
        "profile_sparkline_timeslice_msec": 1000,
        "argobots": {
            "abt_mem_max_num_stacks": 8,
            "abt_thread_stacksize": 2097152,
            "version": "unknown",
            "pools": [
                {
                    "name": "__primary__",
                    "kind": "fifo_wait",
                    "access": "mpmc"
                },
                {
                    "name": "my_rpc_pool",
                    "kind": "fifo",
                    "access": "mpmc"
                }
            ],
            "xstreams": [
                {
                    "name": "__primary__",
                    "cpubind": -1,
                    "affinity": [],
                    "scheduler": {
                        "type": "basic_wait",
                        "pools": [
                       

## Adding providers